In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

In [2]:
raw_csv_data = np.loadtxt('audiobooks.csv', delimiter=',')

In [3]:
unscaled_inputs_all = raw_csv_data[:,1:-1]

In [4]:
targets_all = raw_csv_data[:, -1]

# Balance data set

In [9]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter +=1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0)

# standardize the inputs

In [11]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)


# shuffle data

In [13]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

# Split into training, validation, and test

In [16]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count: train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count: train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

In [17]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_validation', inputs = test_inputs, targets = test_targets)

# Model

In [24]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, 
          train_targets, 
         batch_size=batch_size,
         epochs=max_epochs,
         callbacks = [early_stopping],
         validation_data = (validation_inputs, validation_targets),
        verbose = 2 )

Epoch 1/100
36/36 - 0s - loss: 0.5377 - accuracy: 0.7823 - val_loss: 0.4062 - val_accuracy: 0.8702
Epoch 2/100
36/36 - 0s - loss: 0.3579 - accuracy: 0.8773 - val_loss: 0.3281 - val_accuracy: 0.8747
Epoch 3/100
36/36 - 0s - loss: 0.3146 - accuracy: 0.8852 - val_loss: 0.3067 - val_accuracy: 0.8814
Epoch 4/100
36/36 - 0s - loss: 0.2931 - accuracy: 0.8935 - val_loss: 0.2922 - val_accuracy: 0.8814
Epoch 5/100
36/36 - 0s - loss: 0.2802 - accuracy: 0.8980 - val_loss: 0.2848 - val_accuracy: 0.8881
Epoch 6/100
36/36 - 0s - loss: 0.2701 - accuracy: 0.9011 - val_loss: 0.2791 - val_accuracy: 0.8904
Epoch 7/100
36/36 - 0s - loss: 0.2633 - accuracy: 0.9011 - val_loss: 0.2744 - val_accuracy: 0.8881
Epoch 8/100
36/36 - 0s - loss: 0.2563 - accuracy: 0.9039 - val_loss: 0.2745 - val_accuracy: 0.8949
Epoch 9/100
36/36 - 0s - loss: 0.2546 - accuracy: 0.9047 - val_loss: 0.2724 - val_accuracy: 0.8949
Epoch 10/100
36/36 - 0s - loss: 0.2502 - accuracy: 0.9061 - val_loss: 0.2664 - val_accuracy: 0.8926
Epoch 11/

# Test the model

In [25]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 1ms/step - loss: 0.2815 - accuracy: 0.8996


In [26]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.28. Test accuracy: 89.96%
